## Housing 2º Competition, Kaggle Regression

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

#import set_config
from sklearn import set_config
set_config(transform_output="pandas")

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 300)

In [2]:
# check version of sklearn to verify is bigger than 1.2 to include OrdinalEncoder(categories=ordinal_rankings, encoded_missing_value=-1, handle_unknown='use_encoded_value', unknown_value=-1)

import sklearn
print(sklearn.__version__)

1.3.0


In [3]:
# update sklearn for using root_mean_squared_error
#!pip install --upgrade scikit-learn


In [4]:
# Load data (iteration 6)
url = "https://drive.google.com/file/d/1c5210z-HSQ-Kv4EExbYIjjPq6lT0UYZW/view?usp=drive_link"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
data = pd.read_csv(path)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

## Splitting the data

In [5]:

# Separate the features and target variable
X = data.drop("Id", axis=1).copy()
y = X.pop("SalePrice")

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)


Let's analyze the ordinal categories to determine if any include 'NA' values that should be explicitly added because they differ from missing values:

1. **Alley**: Includes 'NA' for "No alley access". This is a meaningful attribute rather than just missing data.
2. **BsmtQual**: 'NA' signifies "No Basement", which is a different condition than just an unknown basement quality.
3. **BsmtCond**: Similar to `BsmtQual`, 'NA' means "No Basement".
4. **BsmtExposure**: 'NA' again indicates "No Basement".
5. **BsmtFinType1**: 'NA' implies "No Basement".
6. **BsmtFinType2**: 'NA' also implies "No Basement".
7. **FireplaceQu**: 'NA' indicates "No Fireplace", which is distinct from not knowing the quality of the fireplace.
8. **GarageFinish**: 'NA' signifies "No Garage", not just an unobserved or missing garage finish.
9. **GarageQual**: 'NA' signifies "No Garage".
10. **GarageCond**: Similarly, 'NA' here means "No Garage".
11. **PoolQC**: 'NA' stands for "No Pool".
12. **Fence**: 'NA' means "No Fence".

In each of these cases, 'NA' should not simply be treated as a missing value, because it provides explicit information about the absence of a feature, which is valuable in the context of house pricing.


## Selecting types of features

In [6]:
# List of columns containing ordered categorical features

ordinal_categories_column_names = [
    "Street", "Alley", "LotShape", "Utilities", "LandSlope", "ExterQual", "ExterCond",
    "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2",
    "HeatingQC", "KitchenQual", "Functional", "FireplaceQu", "GarageFinish",
    "GarageQual", "GarageCond", "PavedDrive", "PoolQC", "Fence"]


# Separate numeric columns and categorical columns
X_num_columns = X_train.select_dtypes(include='number')
X_cat_columns = X_train.select_dtypes(exclude='number')
X_cat_ordered_columns = X_cat_columns[ordinal_categories_column_names]
X_cat_unordered_columns = X_cat_columns.drop(ordinal_categories_column_names, axis=1)

# Define categories for ordered categorical features

ordinal_rankings = [
    ['Grvl', 'Pave'],  # Street
    ['NA', 'Grvl', 'Pave'],  # Alley
    ['IR3', 'IR2', 'IR1', 'Reg'],  # LotShape
    ["ELO", "NoSeWa", "NoSewr", "AllPub"],  # Utilities
    ["Sev", "Mod", "Gtl"],  # LandSlope
    ['Po', 'Fa', 'TA', 'Gd', 'Ex'],  # ExterQual
    ['Po', 'Fa', 'TA', 'Gd', 'Ex'],  # ExterCond
    ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],  # BsmtQual
    ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],  # BsmtCond
    ['NA', 'No', 'Mn', 'Av', 'Gd'],  # BsmtExposure
    ['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],  # BsmtFinType1
    ['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],  # BsmtFinType2
    ['Po', 'Fa', 'TA', 'Gd', 'Ex'],  # HeatingQC
    ['Po', 'Fa', 'TA', 'Gd', 'Ex'],  # KitchenQual
    ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'],  # Functional
    ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],  # FireplaceQu
    ['NA', 'Unf', 'RFn', 'Fin'],  # GarageFinish
    ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],  # GarageQual
    ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],  # GarageCond
    ['N', 'P', 'Y'],  # PavedDrive
    ['NA', 'Fa', 'TA', 'Gd', 'Ex'],  # PoolQC
    ['NA', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']  # Fence
]


# Create pipelines for preprocessing the data
# (1) For unordered categorical features, fill missing values with "N_A" and perform one-hot encoding
unordered_categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    OneHotEncoder(sparse_output=False, handle_unknown='ignore')
)

# (2) For ordered categorical features, use ordinal encoding with the defined categories
ordered_categoric_pipe = make_pipeline(
    OrdinalEncoder(categories=ordinal_rankings, encoded_missing_value=-1, handle_unknown='use_encoded_value', unknown_value=-1)
)

# (3) For numeric features, fill missing values with the mean
numeric_pipe = make_pipeline(
    SimpleImputer(strategy="mean")
)

# Create a ColumnTransformer to apply the appropriate preprocessing to different feature groups
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipe, X_num_columns.columns),
        ('unordered', unordered_categoric_pipe, X_cat_unordered_columns.columns),
        ('ordered', ordered_categoric_pipe, X_cat_ordered_columns.columns)
    ]
)


In [7]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer())]),
                                 Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'Half...
                                                                 encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1))]),
                                 Index(['Street', 'Alley', 'LotShape', 'Utilities', 'LandSlope', 'ExterQual',
       'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC',
       'Fence'],
      dtype='object'))])

## Understanding the pipeline

We are aiming to have a pipeline that will have this structure:
```python
pipeline = make_pipeline(
    preprocessor,
    MinMaxScaler(),
    SelectKBest(score_func=mutual_info_regression),
    RandomForestRegressor()
)
```

Let's understand what each step does:

1. **preprocessor**: This step will be responsible for encoding the categorical features and imputing missing values.
2. **MinMaxScaler**: This step will scale the features to a range of [0, 1].
3. **SelectKBest**: This step will select the best features based on a scoring function.
4. **RandomForestRegressor**: This step will be the final estimator.




In [8]:
X_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
984,90,RL,75.0,10125,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2009,COD,Normal
777,20,RL,100.0,13350,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal
707,120,RL,48.0,6240,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,12,2009,WD,Normal
598,20,RL,80.0,12984,Pave,NaN,Reg,Bnk,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,3,2006,WD,Normal
874,50,RM,52.0,5720,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2009,WD,Abnorml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,20,RL,78.0,9317,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,3,2007,WD,Normal
1130,50,RL,65.0,7804,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,12,2009,WD,Normal
1294,20,RL,60.0,8172,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Normal
860,50,RL,55.0,7642,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,0,NaN,GdPrv,NaN,0,6,2007,WD,Normal


In [9]:
# 1.Preprocess the training data
X_train_processed = preprocessor.fit_transform(X_train)

X_train_processed

,num__MSSubClass,num__LotFrontage,num__LotArea,num__OverallQual,num__OverallCond,num__YearBuilt,num__YearRemodAdd,num__MasVnrArea,num__BsmtFinSF1,num__BsmtFinSF2,...,ordered__HeatingQC,ordered__KitchenQual,ordered__Functional,ordered__FireplaceQu,ordered__GarageFinish,ordered__GarageQual,ordered__GarageCond,ordered__PavedDrive,ordered__PoolQC,ordered__Fence
984,90.0,75.0,10125.0,5.0,5.0,1977.0,1977.0,0.0,0.0,0.0,...,2.0,3.0,7.0,-1.0,1.0,3.0,3.0,2.0,-1.0,-1.0
777,20.0,100.0,13350.0,5.0,5.0,1974.0,1974.0,0.0,762.0,0.0,...,2.0,2.0,7.0,2.0,1.0,3.0,3.0,2.0,-1.0,3.0
707,120.0,48.0,6240.0,8.0,5.0,2006.0,2006.0,176.0,863.0,0.0,...,4.0,3.0,7.0,4.0,3.0,3.0,3.0,2.0,-1.0,-1.0
598,20.0,80.0,12984.0,5.0,6.0,1977.0,1977.0,459.0,1283.0,147.0,...,4.0,3.0,7.0,3.0,3.0,3.0,3.0,2.0,-1.0,-1.0
874,50.0,52.0,5720.0,5.0,6.0,1941.0,1950.0,0.0,0.0,0.0,...,4.0,2.0,7.0,-1.0,1.0,3.0,3.0,2.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,20.0,78.0,9317.0,6.0,5.0,2006.0,2006.0,0.0,24.0,0.0,...,3.0,3.0,7.0,4.0,2.0,3.0,3.0,2.0,-1.0,-1.0
1130,50.0,65.0,7804.0,4.0,3.0,1928.0,1950.0,0.0,622.0,0.0,...,2.0,3.0,5.0,3.0,1.0,3.0,3.0,2.0,-1.0,3.0
1294,20.0,60.0,8172.0,5.0,7.0,1955.0,1990.0,0.0,167.0,0.0,...,2.0,2.0,7.0,-1.0,1.0,3.0,3.0,0.0,-1.0,-1.0
860,50.0,55.0,7642.0,7.0,8.0,1918.0,1998.0,0.0,0.0,0.0,...,3.0,3.0,7.0,4.0,1.0,3.0,3.0,2.0,-1.0,4.0


In [10]:
pd.set_option('display.max_columns', None)

In [11]:
column_names = X_train_processed.columns
column_names

Index(['num__MSSubClass', 'num__LotFrontage', 'num__LotArea',
       'num__OverallQual', 'num__OverallCond', 'num__YearBuilt',
       'num__YearRemodAdd', 'num__MasVnrArea', 'num__BsmtFinSF1',
       'num__BsmtFinSF2',
       ...
       'ordered__HeatingQC', 'ordered__KitchenQual', 'ordered__Functional',
       'ordered__FireplaceQu', 'ordered__GarageFinish', 'ordered__GarageQual',
       'ordered__GarageCond', 'ordered__PavedDrive', 'ordered__PoolQC',
       'ordered__Fence'],
      dtype='object', length=221)


Certain scaling processes can result in transforming features to have the same variance, like the standard scaler, which changes the standard deviation to 1. However, this is **not** our desired outcome. We need to choose a scaler that preserves variance, and for this purpose, we will use min-max scaling.

In [18]:
# 2.Manually apply MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_processed)
X_train_scaled


,num__MSSubClass,num__LotFrontage,num__LotArea,num__OverallQual,num__OverallCond,num__YearBuilt,num__YearRemodAdd,num__MasVnrArea,num__BsmtFinSF1,num__BsmtFinSF2,num__BsmtUnfSF,num__TotalBsmtSF,num__1stFlrSF,num__2ndFlrSF,num__LowQualFinSF,num__GrLivArea,num__BsmtFullBath,num__BsmtHalfBath,num__FullBath,num__HalfBath,num__BedroomAbvGr,num__KitchenAbvGr,num__TotRmsAbvGrd,num__Fireplaces,num__GarageYrBlt,num__GarageCars,num__GarageArea,num__WoodDeckSF,num__OpenPorchSF,num__EnclosedPorch,num__3SsnPorch,num__ScreenPorch,num__PoolArea,num__MiscVal,num__MoSold,num__YrSold,unordered__MSZoning_C (all),unordered__MSZoning_FV,unordered__MSZoning_RH,unordered__MSZoning_RL,unordered__MSZoning_RM,unordered__LandContour_Bnk,unordered__LandContour_HLS,unordered__LandContour_Low,unordered__LandContour_Lvl,unordered__LotConfig_Corner,unordered__LotConfig_CulDSac,unordered__LotConfig_FR2,unordered__LotConfig_FR3,unordered__LotConfig_Inside,unordered__Neighborhood_Blmngtn,unordered__Neighborhood_Blueste,unordered__Neighborhood_BrDale,unordered__Neighborhood_BrkSide,unordered__Neighborhood_ClearCr,unordered__Neighborhood_CollgCr,unordered__Neighborhood_Crawfor,unordered__Neighborhood_Edwards,unordered__Neighborhood_Gilbert,unordered__Neighborhood_IDOTRR,unordered__Neighborhood_MeadowV,unordered__Neighborhood_Mitchel,unordered__Neighborhood_NAmes,unordered__Neighborhood_NPkVill,unordered__Neighborhood_NWAmes,unordered__Neighborhood_NoRidge,unordered__Neighborhood_NridgHt,unordered__Neighborhood_OldTown,unordered__Neighborhood_SWISU,unordered__Neighborhood_Sawyer,unordered__Neighborhood_SawyerW,unordered__Neighborhood_Somerst,unordered__Neighborhood_StoneBr,unordered__Neighborhood_Timber,unordered__Neighborhood_Veenker,unordered__Condition1_Artery,unordered__Condition1_Feedr,unordered__Condition1_Norm,unordered__Condition1_PosA,unordered__Condition1_PosN,unordered__Condition1_RRAe,unordered__Condition1_RRAn,unordered__Condition1_RRNe,unordered__Condition1_RRNn,unordered__Condition2_Artery,unordered__Condition2_Feedr,unordered__Condition2_Norm,unordered__Condition2_PosA,unordered__Condition2_PosN,unordered__Condition2_RRAe,unordered__Condition2_RRAn,unordered__Condition2_RRNn,unordered__BldgType_1Fam,unordered__BldgType_2fmCon,unordered__BldgType_Duplex,unordered__BldgType_Twnhs,unordered__BldgType_TwnhsE,unordered__HouseStyle_1.5Fin,unordered__HouseStyle_1.5Unf,unordered__HouseStyle_1Story,unordered__HouseStyle_2.5Fin,unordered__HouseStyle_2.5Unf,unordered__HouseStyle_2Story,unordered__HouseStyle_SFoyer,unordered__HouseStyle_SLvl,unordered__RoofStyle_Flat,unordered__RoofStyle_Gable,unordered__RoofStyle_Gambrel,unordered__RoofStyle_Hip,unordered__RoofStyle_Mansard,unordered__RoofStyle_Shed,unordered__RoofMatl_ClyTile,unordered__RoofMatl_CompShg,unordered__RoofMatl_Metal,unordered__RoofMatl_Roll,unordered__RoofMatl_Tar&Grv,unordered__RoofMatl_WdShake,unordered__RoofMatl_WdShngl,unordered__Exterior1st_AsbShng,unordered__Exterior1st_AsphShn,unordered__Exterior1st_BrkComm,unordered__Exterior1st_BrkFace,unordered__Exterior1st_CBlock,unordered__Exterior1st_CemntBd,unordered__Exterior1st_HdBoard,unordered__Exterior1st_ImStucc,unordered__Exterior1st_MetalSd,unordered__Exterior1st_Plywood,unordered__Exterior1st_Stone,unordered__Exterior1st_Stucco,unordered__Exterior1st_VinylSd,unordered__Exterior1st_Wd Sdng,unordered__Exterior1st_WdShing,unordered__Exterior2nd_AsbShng,unordered__Exterior2nd_AsphShn,unordered__Exterior2nd_Brk Cmn,unordered__Exterior2nd_BrkFace,unordered__Exterior2nd_CBlock,unordered__Exterior2nd_CmentBd,unordered__Exterior2nd_HdBoard,unordered__Exterior2nd_ImStucc,unordered__Exterior2nd_MetalSd,unordered__Exterior2nd_Other,unordered__Exterior2nd_Plywood,unordered__Exterior2nd_Stone,unordered__Exterior2nd_Stucco,unordered__Exterior2nd_VinylSd,unordered__Exterior2nd_Wd Sdng,unordered__Exterior2nd_Wd Shng,unordered__MasVnrType_BrkCmn,unordered__MasVnrType_BrkFace,unordered__MasVnrType_N_A,unordered__MasVnrType_Stone,unordered__Foundation_BrkTil,

In [19]:
X_train_scaled.shape

(1241, 221)

- If you expect or know that your data exhibits mostly linear relationships, or you're working with relatively simple models and want to keep computational costs low, `f_regression` might be the more appropriate choice.
    
- If you're dealing with complex data where features might have nonlinear effects on the target, or when you don't want to miss out on any potential relationships (regardless of their nature), `mutual_info_regression` can be more effective, despite its higher computational demand.

In [15]:
# 4. Apply SelectKBest
selector = SelectKBest(score_func=mutual_info_regression, k=10)  # Choose k as desired
X_train_kbest = selector.fit_transform(X_train_scaled, y_train)

# Get selected feature indices from SelectKBest
selected_indices = selector.get_support(indices=True)

# Translate selected indices to feature names
feature_names_processed= preprocessor.get_feature_names_out()
selected_feature_names = [feature_names_processed[i] for i in selected_indices]

print("Top features selected by SelectKBest:", selected_feature_names)

X_train_kbest

Top features selected by SelectKBest: ['num__OverallQual', 'num__YearBuilt', 'num__TotalBsmtSF', 'num__1stFlrSF', 'num__GrLivArea', 'num__GarageCars', 'num__GarageArea', 'ordered__ExterQual', 'ordered__BsmtQual', 'ordered__KitchenQual']


,num__OverallQual,num__YearBuilt,num__TotalBsmtSF,num__1stFlrSF,num__GrLivArea,num__GarageCars,num__GarageArea,ordered__ExterQual,ordered__BsmtQual,ordered__KitchenQual
984,0.444444,0.760870,0.000000,0.222120,0.263753,0.50,0.380113,0.333333,0.000000,0.666667
777,0.444444,0.739130,0.141408,0.128499,0.105501,0.50,0.310296,0.333333,0.666667,0.333333
707,0.777778,0.971014,0.216694,0.227168,0.186511,0.50,0.387870,0.666667,0.833333,0.666667
598,0.444444,0.760870,0.234043,0.301285,0.247362,0.50,0.437941,0.333333,0.833333,0.666667
874,0.444444,0.500000,0.110638,0.078476,0.150151,0.25,0.141044,0.333333,0.666667,0.333333
...,...,...,...,...,...,...,...,...,...,...
1095,0.555556,0.971014,0.215057,0.224874,0.184627,0.50,0.310296,0.666667,0.833333,0.666667
1130,0.333333,0.405797,0.183633,0.228086,0.310286,0.50,0.406206,0.333333,0.666667,0.666667
1294,0.444444,0.601449,0.141408,0.121615,0.099849,0.50,0.403385,0.333333,0.666667,0.333333
860,0.666667,0.333333,0.149264,0.132630,0.205727,0.25,0.152327,0.666667,0.666667,0.666667


In [22]:
from sklearn.feature_selection import f_regression
# 4. Apply SelectKBest
selector = SelectKBest(score_func=f_regression, k=10)  # Choose k as desired
X_train_kbest = selector.fit_transform(X_train_scaled, y_train)

# Get selected feature indices from SelectKBest
selected_indices = selector.get_support(indices=True)

# Translate selected indices to feature names
selected_feature_names = [feature_names_processed[i] for i in selected_indices]

print("Top features selected by SelectKBest:", selected_feature_names)

X_train_kbest

Top features selected by SelectKBest: ['num__OverallQual', 'num__TotalBsmtSF', 'num__1stFlrSF', 'num__GrLivArea', 'num__FullBath', 'num__GarageCars', 'num__GarageArea', 'ordered__ExterQual', 'ordered__BsmtQual', 'ordered__KitchenQual']


,num__OverallQual,num__TotalBsmtSF,num__1stFlrSF,num__GrLivArea,num__FullBath,num__GarageCars,num__GarageArea,ordered__ExterQual,ordered__BsmtQual,ordered__KitchenQual
984,0.444444,0.000000,0.222120,0.263753,0.666667,0.50,0.380113,0.333333,0.000000,0.666667
777,0.444444,0.141408,0.128499,0.105501,0.333333,0.50,0.310296,0.333333,0.666667,0.333333
707,0.777778,0.216694,0.227168,0.186511,0.666667,0.50,0.387870,0.666667,0.833333,0.666667
598,0.444444,0.234043,0.301285,0.247362,0.666667,0.50,0.437941,0.333333,0.833333,0.666667
874,0.444444,0.110638,0.078476,0.150151,0.333333,0.25,0.141044,0.333333,0.666667,0.333333
...,...,...,...,...,...,...,...,...,...,...
1095,0.555556,0.215057,0.224874,0.184627,0.666667,0.50,0.310296,0.666667,0.833333,0.666667
1130,0.333333,0.183633,0.228086,0.310286,0.666667,0.50,0.406206,0.333333,0.666667,0.666667
1294,0.444444,0.141408,0.121615,0.099849,0.333333,0.50,0.403385,0.333333,0.666667,0.333333
860,0.666667,0.149264,0.132630,0.205727,0.333333,0.25,0.152327,0.666667,0.666667,0.666667


When using cross-validation with Scikit-Learn, the scoring mechanism is designed with a convention that a bigger (higher) score indicates a better model 📈. However, with error metrics such as Root Mean Absolute Error (RMAE) or Root Mean Squared Log Error (RMSLE), we are in a situation where a good model is indicated by a lower value since we want to minimize error, not maximize it 📉. Scikit-Learn addresses this by using negated versions of these metrics, like 'neg_root_mean_squared_log_error'.

Using the negated version means that the error is multiplied by -1, essentially flipping the sign 🔁. So, during the cross-validation process, when Scikit-Learn tries to maximize the scoring function, it's effectively minimizing the original error metric.

Here's a straightforward example:

- **Model 1** has an RMSLE of approximately 0.00576 📊.
- **Model 2** has an RMSLE of approximately 0.00235 📈.

When these MSLE scores are used in a cross-validation context within Scikit-Learn, we use their negated values:

- **Model 1** has a negated RMSLE of approximately -0.00576 ➖.
- **Model 2** has a negated RMSLE of approximately -0.00235 ➖.

In cross-validation, we look for the model that has the "highest" score because Scikit-Learn's convention is "higher is better." Since these are negated errors, a higher value (closer to zero) means a lower actual error. Therefore, Model 2, with a negated RMSLE of -0.00235, will be determined correctly as better than Model 1 with -0.00576 ✅.

Additionally, when you use cross-validation with 'neg_root_mean_squared_log_error' as your scoring function, you are being consistent with the competition's evaluation metric. This consistency is critical because it ensures that the model selection and tuning process during development are in direct correlation with the performance metric that determines your final position in the competition 🏆.

In [24]:

# Define the pipeline
pipeline = make_pipeline(
    preprocessor,
    MinMaxScaler(),
    SelectKBest(score_func=mutual_info_regression), # or score_func=f_regression
    RandomForestRegressor()
)

# Define the hyperparameter grid for RandomizedSearchCV
param_grid = {
    "columntransformer__num__simpleimputer__strategy": ["mean", "median"],
    "randomforestregressor__n_estimators": range(50, 400, 50),
    "randomforestregressor__max_depth": range(1, 15, 2),
    "randomforestregressor__min_samples_split": range(2, 20, 2),
    "selectkbest__k": range(1, 70, 2)
}

# Perform RandomizedSearchCV for hyperparameter tuning
search = RandomizedSearchCV(pipeline,
                            param_distributions=param_grid,
                            n_iter=30,
                            cv=5,
                            scoring="neg_root_mean_squared_log_error",
                            verbose=1,
                            n_jobs=-1)

# Fit the model to the training data
search.fit(X_train, y_train)

#creating the dictionary to store the average scores
avg_scores = {"rforest" : search.best_score_}

avg_scores

InvalidParameterError: The 'scoring' parameter of RandomizedSearchCV must be a str among {'roc_auc_ovo', 'adjusted_mutual_info_score', 'jaccard_samples', 'average_precision', 'neg_mean_poisson_deviance', 'mutual_info_score', 'balanced_accuracy', 'explained_variance', 'neg_log_loss', 'roc_auc', 'f1_samples', 'f1_macro', 'precision_samples', 'neg_negative_likelihood_ratio', 'recall_weighted', 'neg_root_mean_squared_error', 'accuracy', 'neg_mean_absolute_percentage_error', 'neg_median_absolute_error', 'precision_macro', 'r2', 'recall', 'recall_samples', 'neg_mean_squared_log_error', 'completeness_score', 'max_error', 'homogeneity_score', 'jaccard_macro', 'neg_mean_absolute_error', 'f1_weighted', 'top_k_accuracy', 'neg_mean_gamma_deviance', 'v_measure_score', 'neg_mean_squared_error', 'normalized_mutual_info_score', 'roc_auc_ovr_weighted', 'f1_micro', 'recall_micro', 'f1', 'rand_score', 'precision_micro', 'jaccard', 'jaccard_micro', 'roc_auc_ovr', 'recall_macro', 'positive_likelihood_ratio

However, feel free to use other metrics to evaluate your model's performance during development. Metrics like R2 and RMSE will allow you to understand your model's behavior and performance in a more human-readable way 🚀.

In [23]:

# Make predictions on training and test sets
y_train_pred = search.predict(X_train)
y_test_pred = search.predict(X_test)

# Calculate R-squared scores and root mean squared errors for training and test sets
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

rmse_train = root_mean_squared_error(y_train, y_train_pred)
rmse_test = root_mean_squared_error(y_test, y_test_pred)

# Create a DataFrame to store the model evaluation results
score_data = {
    'RF_R2_Score': [r2_train, r2_test],
    'RF_RMSE': [rmse_train, rmse_test]
}

index_values = ['Train', 'Test']

df_scores = pd.DataFrame(score_data, index=index_values)

# Display the model evaluation results
df_scores

NotFittedError: This RandomizedSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

## Gradient Boosting

XGBoost
Extreme Gradient Boost (XGB) is a boosting algorithm that uses the gradient boosting framework; where gradient descent algorithm is employed to minimize the errors in the sequential model. It improves on the gradient boosting framework with faster execution speed and improved performance.

![image.png](https://miro.medium.com/max/1400/1*QJZ6W-Pck_W7RlIDwUIN9Q.jpeg)

In [ ]:
xgb_pipeline = make_pipeline(
    preprocessor,
    MinMaxScaler(),
    SelectKBest(score_func=mutual_info_regression),
    XGBRegressor()
)

xgb_param_grid = {
    "columntransformer__num__simpleimputer__strategy": ["mean", "median"],
    "selectkbest__k": range(1, 70, 2),
    "xgbregressor__learning_rate": [0.01, 0.1, 0.15, 0.3, 0.5],
    "xgbregressor__n_estimators": [100, 500, 1000, 2000, 3000],
    "xgbregressor__max_depth": [3, 6, 9],
    "xgbregressor__min_child_weight": [1, 5, 10, 20],
    "xgbregressor__reg_alpha": [0.001, 0.01, 0.1],
    "xgbregressor__reg_lambda": [0.001, 0.01, 0.1]
}

xgb_search = RandomizedSearchCV(xgb_pipeline,
                            param_distributions=xgb_param_grid,
                            n_iter=30,
                            cv=5,
                            scoring="neg_root_mean_squared_log_error",
                            verbose=1,
                            n_jobs=-1)

xgb_search.fit(X_train, y_train)



print("Best Parameters:", xgb_search.best_params_)

avg_scores["xgb"] = xgb_search.best_score_
avg_scores


In [ ]:

# Make predictions on training and test sets
y_train_pred = xgb_search.predict(X_train)
y_test_pred = xgb_search.predict(X_test)

# Calculate R-squared scores and root mean squared errors for training and test sets
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

rmse_train = root_mean_squared_error(y_train, y_train_pred)
rmse_test = root_mean_squared_error(y_test, y_test_pred)

# Create a DataFrame to store the model evaluation results
score_data = {
    'XGB_R2_Score': [r2_train, r2_test],
    'XGB_RMSE': [rmse_train, rmse_test]
}

index_values = ['Train', 'Test']

# Concatanate the results to the previous DataFrame
df_scores = pd.concat([df_scores, pd.DataFrame(score_data, index=index_values)], axis=1)

# Display the model evaluation results
df_scores

https://neptune.ai/blog/lightgbm-parameters-guide
LightBGM is another gradient boosting framework developed by Microsoft that is based on decision tree algorithm, designed to be efficient and distributed. Some of the advantages of implementing LightBGM compared to other boosting frameworks include:

1. Faster training speed and higher efficiency (use histogram based algorithm i.e it buckets continuous feature values into discrete bins which fasten the training procedure)
2. Lower memory usage (Replaces continuous values to discrete bins which result in lower memory usage)
3. Better accuracy
4. Support of parallel and GPU learning
5. Capable of handling large-scale data (capable of performing equally good with large datasets with a significant reduction in training time as compared to XGBOOST)

LightGBM splits the tree leaf wise with the best fit whereas other boosting algorithms split the tree depth wise or level wise rather than leaf-wise. This leaf-wise algorithm reduces more loss than the level-wise algorithm, hence resulting in much better accuracy which can rarely be achieved by any of the existing boosting algorithms.

![image.png](https://rohitgr7.github.io/content/images/2019/03/Screenshot-from-2019-03-27-23-09-47-1.png)

In [ ]:
# LightGBM Pipeline
lgbm_pipeline = make_pipeline(
    preprocessor,
    MinMaxScaler(),
    SelectKBest(score_func=mutual_info_regression),
    LGBMRegressor()
)


lgbm_param_grid = {
    'columntransformer__num__simpleimputer__strategy': ['mean', 'median'],
    'lgbmregressor__num_leaves': [15, 31, 63],
    'lgbmregressor__max_depth': [5, 10, -1],  # Include no limit (-1) as an option
    'lgbmregressor__learning_rate': [0.005, 0.01, 0.05, 0.1],
    'lgbmregressor__n_estimators': [100, 300, 500, 1000, 1500],
    'lgbmregressor__reg_alpha': [0, 0.001, 0.01, 0.1],
    'lgbmregressor__reg_lambda': [0, 0.001, 0.01, 0.1],
    'lgbmregressor__feature_fraction': [0.5, 0.6, 0.7, 0.8],  # Or 'colsample_bytree' if appropriate
    'lgbmregressor__min_child_samples': [20, 30, 40, 50],
    "selectkbest__k": range(50, 70, 2) # 'all' to test without feature selection as well

}


# Hyperparameter Tuning with RandomizedSearchCV
lgbm_search = RandomizedSearchCV(lgbm_pipeline,
                            param_distributions=lgbm_param_grid,
                            n_iter=40,
                            cv=5,
                            scoring="neg_root_mean_squared_log_error",
                            verbose=1,
                            n_jobs=-1)


lgbm_search.fit(X_train, y_train)



In [ ]:
print("Best Parameters:", lgbm_search.best_params_)
avg_scores["lgbm"] = lgbm_search.best_score_
avg_scores

In [ ]:
# Make predictions on training and test sets
y_train_pred = lgbm_search.predict(X_train)
y_test_pred = lgbm_search.predict(X_test)

# Calculate R-squared scores and root mean squared errors for training and test sets
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

rmse_train = root_mean_squared_error(y_train, y_train_pred)
rmse_test = root_mean_squared_error(y_test, y_test_pred)

# Create a DataFrame to store the model evaluation results
score_data = {
    'Lgbm_R2_Score': [r2_train, r2_test],
    'Lgbm_RMSE': [rmse_train, rmse_test]
}

index_values = ['Train', 'Test']

# Concatanate the results to the previous DataFrame
df_scores = pd.concat([df_scores, pd.DataFrame(score_data, index=index_values)], axis=1)

# Display the model evaluation results
df_scores

## Import kaggle data

In [ ]:
competition_data = pd.read_csv("kaggle_test.csv")
competition_ids = competition_data.pop('Id')
competition_ids

In [ ]:
my_submission = pd.DataFrame({"Id": competition_ids})
my_submission

In [ ]:
my_submission["SalePrice"] = lgbm_search.predict(competition_data)
my_submission.head(20)

In [ ]:
my_submission.to_csv("lgbm_submission.csv", index=False)